![TQ42%20Banner.png](attachment:TQ42%20Banner.png)

# Welcome to TQ42

## Example: Using Custom Quantum Layers

## ✨ Introduction 
This notebook demonstrates how to create and use a custom quantum layer within a classical machine learning model using the TQ42 SDK. For more details, please see the [developer documentation](https://docs.tq42.com/en/latest/index.html).

We'll tackle a regression problem with the Boston House Price dataset, predicting house prices based on property and neighborhood features. 

## 🎯 Objective

Applying a custom quantum layer within a classical architecture involves:
1. Preparing and connecting a training dataset
2. Designing a custom quantum layer by creating and ordering the gates from our TQnet library
3. Integrating the custom quantum layer into a classical neural network model
4. Creating and monitoring an experiment run to train the model using the data

# 1. Import the TQ42 API

In [ ]:
# Import packages

from google.protobuf.json_format import MessageToDict
from tq42.client import TQ42Client
from tq42.experiment_run import ExperimentRun
from tq42.compute import HardwareProto
from tq42.algorithm import (
    TrainDataProcessingParametersProto,
    OptimProto,
    LossFuncProto,
    DatasetStorageInfoProto,
    GenericMLTrainMetadataProto,
    GenericMLTrainParametersProto,
    Layer,
    ClassicalDenseLayer,
    MLModelType,
    TrainModelInfoProto,
    MLTrainInputsProto,
    AlgorithmProto,
    MeasureProto,
    CustomQuantumLayer,
    CnotGate,
    HadamardGate,
    VariationalGate,
    EncodingGate,
    MeasurementGate,
    Gate
)


# 2. Create a client

In [ ]:
# Connect to the cloud service
# https://terraquantum.io

with TQ42Client() as client:
    client.login()

# 3. Select an organization, project and experiment

In [ ]:
# List the organizations available to you and select one

with TQ42Client() as client: 
    org_list = list_all_organizations(client=client)
    
org=None
def f(x):
    global org
    for o in org_list:
        if o.data.name == x:
            org = o
            print(f"Using organization {x}, {org.data.id}")
            return
        
interact(f, x=[o.data.name for o in org_list]);

In [ ]:
# List the projects within that organization and select one
   
with TQ42Client() as client: 
    proj_list = list_all_projects(client=client, organization_id=org.id)
    
proj=None
def f(x):
    global proj
    for p in proj_list:
        if p.data.name == x:
            proj = p
            print(f"Using project {x}, {proj.data.id}")
            return
        
interact(f, x=[p.data.name for p in proj_list]);

In [ ]:
# List the experiments within that project and select one

with TQ42Client() as client: 
    exp_list = list_all_experiments(client=client, project_id=proj.id)
    
exp=None
def f(x):
    global exp
    for e in exp_list:
        if e.data.name == x:
            exp = e
            print(f"Using experiment {x}, {exp.data.id}")
            return
        
interact(f, x=[e.data.name for e in exp_list]);

# 4. List Datasets Available in the Project

In [ ]:
# List datasets in the project you can use to train your model
# You'll provide the id for one of these datasets along with the input and output columns when you train the model below. If you don't know the input/output columns for the dataset, it's best to upload a new one.

with TQ42Client() as client:
    # datasets = list_all(client=client, project_id="ENTER_PROJECT_ID_HERE")
    datasets = list_all(client=client, project_id="347826c3-5b62-4bd7-ad06-cfd11b9e4746")

# 5. (Optional) Upload a Dataset

In [ ]:
# If you do not already have a dataset in that project, upload a dataset from a cloud storage bucket
# Note: The URL format is gs://[id_of_bucket].

from tq42.client import TQ42Client
from tq42.dataset import Dataset, DatasetSensitivityProto

with TQ42Client() as client:
    dataset = Dataset.create(
        client=client,
        name="<NAME_OF_THE_NEW_DATASET>",
        description="<DESCRIPTION_OF_THE_NEW_DATASET>",
        url="gs://<THIS_IS_YOUR_BUCKET_URL>",
        sensitivity=DatasetSensitivityProto.SENSITIVE,
        project_id="<PROJECT_ID>",
    )
    print(dataset.id)

# 6. Build and Train the Custom-Hybrid Network

Verify your experiment run details

In [ ]:
print(f"Running experiment within: Org {org.data.name, org.id}, Proj {proj.data.name, proj.id} and Exp {exp.data.name, exp.id}`")

In [ ]:
# Create and design the Custom Quantum Layer
custom_quantum_layer_msg = CustomQuantumLayer(
    num_qubits=2,
    gates=[
        # Apply Hadamard gates to create superposition
        Gate(hadamard=HadamardGate(wire=0)),
        Gate(hadamard=HadamardGate(wire=1)),
        # Apply a variational gate for parameter optimization
        Gate(variational=VariationalGate(wire=0, rotation=MeasureProto.X)),
        Gate(
            # Encode classical data into the quantum circuit
            encoding=EncodingGate(wire=1, rotation=MeasureProto.Y, feature_id=0) # Encode classical feature 0 as Y rotation on qubit 1
        ),
        # Apply entanglement between qubits
        Gate(cnot=CnotGate(wire1=0, wire2=1)), # CNOT gate with control qubit 0 and target qubit 1
        # Apply another variational gate
        Gate(variational=VariationalGate(wire=1, rotation=MeasureProto.X)),
        # Measure the qubits
        Gate(measurement=MeasurementGate(wire=0, pauli=MeasureProto.X)),
        Gate(measurement=MeasurementGate(wire=1, pauli=MeasureProto.X)),
    ],
)
# Create a classical neural network and integrate the CQ layer
env_msg = GenericMLTrainMetadataProto(
    parameters=GenericMLTrainParametersProto(
        model_type=MLModelType.MLP,
        # Add and customize layers here
        layers=[
            Layer(custom_quantum_layer=custom_quantum_layer_msg),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=1, bias=True)),
        ],
        # WHY ONLY 1 EPOCH? !!!!!!!!!!!!!!!!         !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!         !!!!!!!!!!!!!!!!!!!!!!!!!!
        num_epochs=1,
        k_fold=1,
        batch_size=128,
        learning_rate=0.01,
        optim=OptimProto.ADAM,
        loss_func=LossFuncProto.MAE,
        train_model_info=TrainModelInfoProto(
            # Provide a unique name to identify your trained model
            name="local_test",
            # Add a brief description to help users understand the purpose or functionality of this trained model
            description="a_description",
        ),
        # Specify the input and output columns for your dataset
        data_processing_parameters=TrainDataProcessingParametersProto(
            input_columns=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
            output_columns=[13]
            # input_columns=[0, 1, 2, 3], output_columns=[4], timestamp_columns=[]
        ),
    ),
    inputs=MLTrainInputsProto(
        # Provide the specific dataset storage ID of the data you uploaded to TQ42
        data=DatasetStorageInfoProto(storage_id="STORAGE ID")
    ),
)

with TQ42Client() as client:
    client.login()
    run = ExperimentRun.create(
        client=client,
        algorithm=AlgorithmProto.GENERIC_ML_TRAIN,
        # Provide the specific experiement ID your are running on TQ42
        experiment_id="b2004690-2a4b-479e-bafe-13a475bb0d69",
        compute=HardwareProto.SMALL,
        parameters=MessageToDict(env_msg, preserving_proto_field_name=True)
    )

    print(run.data)

    # Poll for the results
    result = run.poll()

    print(result.data)